In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('drive/MyDrive/Colab Notebooks/hackjun2022/train_wn75k28.csv')

In [ ]:
df.head()

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12,buy
0,1,2021-01-01,1,2,2.0,2020-09-24,0,0,0,0,0,0,1,1,0,0,0,0,0
1,2,2021-01-01,2,1,2.0,2020-09-19,1,0,1,0,0,0,1,0,0,0,0,0,0
2,3,2021-01-01,9,3,3.0,2021-08-11,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,2021-01-01,6,7,2.0,2017-10-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,2021-01-01,4,6,NaN,2020-06-08,0,0,0,0,0,0,1,0,0,0,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, stratify = df['buy'], shuffle = 1000, random_state = 0)

In [ ]:
train.shape, test.shape

((31328, 19), (7833, 19))

In [ ]:
train_y = train['buy']
train_x = train.drop('buy', axis=1)
test_y = test['buy']
test_x = test.drop('buy', axis=1)

In [ ]:
def preprocess(data):
    data[['created_at', 'signup_date']] = data[['created_at', 'signup_date']].apply(pd.to_datetime)
    max_date = data['created_at'].max()
    data['created_days'] = data['created_at'].apply(lambda x: (max_date-x).days)
    data['is_signed'] = data['signup_date'].apply(lambda x: 0 if pd.isna(x) else 1)
    data['signed_days'] = data.apply(lambda x: (x['created_at']-x['signup_date']).days, axis=1).fillna(-1)
    
    float_cols = data.select_dtypes(np.float64)
    float_cols.fillna(0, inplace=True)
    float_cols = float_cols.apply(np.int64)
    
    num_cols = data.select_dtypes(np.int64)
    num_cols.drop('id', axis=1, inplace=True)
    num_cols.fillna(0, inplace=True)
    
    features_data = pd.concat([num_cols, float_cols], axis=1)
    features = features_data.columns
    
    return features_data

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    class_res = classification_report(predictions, test_labels)
    print(class_res)
    return class_res

In [ ]:
train_f = preprocess(train_x)
features = train_f.columns
scaler = StandardScaler()
train_f = scaler.fit_transform(train_f)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train_f

array([[ 1.00142637,  0.20723316, -0.78655703, ..., -1.25948888,
        -0.83701275, -0.5554422 ],
       [ 1.00142637,  0.59088761, -0.78655703, ..., -1.25948888,
        -0.83701275, -0.5554422 ],
       [-0.15313787,  1.3581965 , -0.78655703, ..., -1.25948888,
        -0.00454393, -0.5554422 ],
       ...,
       [-1.3077021 ,  0.97454206, -0.78655703, ..., -1.25948888,
        -0.83701275, -0.5554422 ],
       [-1.3077021 , -0.56007573, -0.78655703, ...,  0.79397287,
         1.66039371, -0.38790229],
       [-0.44177893, -0.56007573, -0.78655703, ...,  0.79397287,
        -0.83701275, -0.77683424]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()
classifier.fit(train_f, train_y)

RandomForestClassifier()

In [ ]:
test_f = preprocess(test_x)
test_f = scaler.transform(test_f)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
result = evaluate(classifier, test_f, test_y)


              precision    recall  f1-score   support

           0       1.00      0.98      0.99      7590
           1       0.56      0.91      0.69       243

    accuracy                           0.97      7833
   macro avg       0.78      0.95      0.84      7833
weighted avg       0.98      0.97      0.98      7833



In [ ]:
from sklearn.svm import SVC
classifier = SVC()
classifier.fit(train_f, train_y)

SVC()

In [ ]:
result = evaluate(classifier, test_f, test_y)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      7599
           1       0.55      0.94      0.70       234

    accuracy                           0.98      7833
   macro avg       0.78      0.96      0.84      7833
weighted avg       0.98      0.98      0.98      7833



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
classifier.fit(train_f, train_y)

GradientBoostingClassifier()

In [ ]:
result = evaluate(classifier, test_f, test_y)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      7588
           1       0.57      0.93      0.70       245

    accuracy                           0.98      7833
   macro avg       0.78      0.95      0.85      7833
weighted avg       0.98      0.98      0.98      7833



In [ ]:
from sklearn.ensemble import AdaBoostClassifier
classifier = AdaBoostClassifier()
classifier.fit(train_f, train_y)

AdaBoostClassifier()

In [ ]:
result = evaluate(classifier, test_f, test_y)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      7591
           1       0.57      0.94      0.71       242

    accuracy                           0.98      7833
   macro avg       0.78      0.96      0.85      7833
weighted avg       0.98      0.98      0.98      7833



In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifier = QuadraticDiscriminantAnalysis()
classifier.fit(train_f, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

In [ ]:
result = evaluate(classifier, test_f, test_y)

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      6537
           1       0.56      0.17      0.26      1296

    accuracy                           0.84      7833
   macro avg       0.71      0.57      0.59      7833
weighted avg       0.81      0.84      0.80      7833



In [ ]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier()
classifier.fit(train_f, train_y)

MLPClassifier()

In [ ]:
result = evaluate(classifier, test_f, test_y)

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      7547
           1       0.59      0.83      0.69       286

    accuracy                           0.97      7833
   macro avg       0.79      0.90      0.84      7833
weighted avg       0.98      0.97      0.98      7833



In [ ]:
test_df = pd.read_csv('drive/MyDrive/Colab Notebooks/hackjun2022/test_Wf7sxXF.csv')

In [ ]:
test_df.head()

,id,created_at,campaign_var_1,campaign_var_2,products_purchased,signup_date,user_activity_var_1,user_activity_var_2,user_activity_var_3,user_activity_var_4,user_activity_var_5,user_activity_var_6,user_activity_var_7,user_activity_var_8,user_activity_var_9,user_activity_var_10,user_activity_var_11,user_activity_var_12
0,39162,2022-01-01,2,2,NaN,2021-08-17,1,1,0,0,0,1,0,0,0,0,1,0
1,39163,2022-01-01,4,7,3.0,2020-05-21,1,0,0,0,0,0,0,1,0,0,0,0
2,39164,2022-01-01,8,7,NaN,NaN,0,0,0,0,1,1,0,0,0,0,0,0
3,39165,2022-01-01,9,8,2.0,2020-06-22,0,0,0,0,1,1,1,0,0,0,2,0
4,39166,2022-01-01,4,5,2.0,2021-03-10,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data = preprocess(test_df)
test_data = scaler.transform(test_data)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
classifier = AdaBoostClassifier()
classifier.fit(train_f, train_y)

AdaBoostClassifier()

In [ ]:
preds = classifier.predict(test_data)
preds

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
submit_df = pd.DataFrame({'id': test_df['id'], 'buy': preds})
submit_df.head()

,id,buy
0,39162,1
1,39163,0
2,39164,0
3,39165,1
4,39166,0


In [47]:
submit_df.to_csv("submission.csv", index=False)